## Question 1

✅Created twitter account with MSU email, saved all information in a config.ini file.

## Question 2

In [2]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API
import pandas as pd
import configparser
import numpy as np

✅Keys.

In [23]:

config = configparser.ConfigParser(interpolation=None)
config.read('config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

✅Set authorize related variables.

In [24]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [25]:
publictweets = api.home_timeline()

In [37]:
user = 'CDCgov'
tm = 'extended'
limit = 30

tweets = tweepy.Cursor(api.user_timeline, screen_name = user, tweet_mode = tm).items(limit)

✅Loop through the df and store the informations in a new df.

In [38]:
df = []
columns = ["Timeline","User","tweet"]

for i in tweets:
    df.append([i.created_at,i.user.screen_name,i.full_text])
    
data = pd.DataFrame(df,columns=columns)

In [40]:
data.head()

,Timeline,User,tweet
0,2022-09-28 21:12:47+00:00,CDCgov,RT @CDCEnvironment: Be safe if the power goes ...
1,2022-09-28 20:30:04+00:00,CDCgov,#DYK you can get either Moderna or Pfizer-BioN...
2,2022-09-28 20:20:31+00:00,CDCgov,RT @CDCEnvironment: Your area may be affected ...
3,2022-09-28 19:46:21+00:00,CDCgov,RT @988Lifeline: As #SPM22 comes to an end rem...
4,2022-09-28 19:03:59+00:00,CDCgov,RT @CDCDirector: CDC continues to work on a po...


In [41]:
tt = data['tweet'].to_numpy()

np.savetxt("cdcgov.txt", tt, fmt='%s', encoding='utf-8')

## Question 3

In [30]:
import time
import json

In [31]:
class MyListener(tweepy.StreamingClient):
    def __init__(self, bearer_token, time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        self.outFile = open('politics.json', 'a', encoding='utf-8')
        super(MyListener, self).__init__(bearer_token)
    
    def on_data(self, data):
        if (time.time() - self.start_time) < self.limit:
            self.outFile.write(data.decode('utf-8'))
            self.outFile.write('\n')
        else:
            self.outFile.close()
            self.disconnect()

    def on_error(self, status):
        print(status)

In [32]:

bearer_token = config['twitter']['Bearer_token']

In [33]:
streaming_client = MyListener(bearer_token)
streaming_client.add_rules(tweepy.StreamRule('"politics"'))
streaming_client.filter()
streaming_client.get_rules()

Stream connection closed by Twitter


Response(data=[StreamRule(value='politics', tag=None, id='1575269469286502400'), StreamRule(value='"politics"', tag=None, id='1575269997672366083')], includes={}, errors=[], meta={'sent': '2022-09-29T00:16:41.692Z', 'result_count': 2})